In [5]:
import pandas as pd
from pathlib import Path

INPUT_CSV = "survey_data_updated 5.csv"   # <-- ajusta ruta
OUT_DIR   = Path("export_top10"); OUT_DIR.mkdir(exist_ok=True)

# ------- alias por categoría (insensible a mayúsc/mínusc) -------
ALIASES = {
    # actuales (WorkedWith = uso actual)
    "LanguageHaveWorkedWith": ["LanguageHaveWorkedWith","LanguagesHaveWorkedWith","Language Worked With","Language"],
    "DatabaseHaveWorkedWith": ["DatabaseHaveWorkedWith","DatabasesHaveWorkedWith","Database Worked With","Database"],
    "PlatformHaveWorkedWith": ["PlatformHaveWorkedWith","PlatformsHaveWorkedWith","Platform Worked With","Platform"],
    # usamos tus nombres reales para WebFrame
    "WebframeHaveWorkedWith": ["WebframeHaveWorkedWith","WebFrameHaveWorkedWith","Webframe Worked With","Web Framework"],
    # futuros (WantToWorkWith = deseo próximo año) por si los necesitas
    "LanguageWantToWorkWith": ["LanguageWantToWorkWith","LanguagesWantToWorkWith","Language Want To Work With"],
    "DatabaseWantToWorkWith": ["DatabaseWantToWorkWith","DatabasesWantToWorkWith","Database Want To Work With"],
    "PlatformWantToWorkWith": ["PlatformWantToWorkWith","PlatformsWantToWorkWith","Platform Want To Work With"],
    "WebframeWantToWorkWith": ["WebframeWantToWorkWith","WebFrameWantToWorkWith","Webframe Want To Work With"]
}

# ------- utilidades -------
def resolve_col(df, candidates):
    cols = [c.strip() for c in df.columns]
    # match exact (case-insensitive)
    for a in candidates:
        for c in cols:
            if c.lower() == a.lower():
                return c
    # match parcial
    for a in candidates:
        for c in cols:
            if a.lower() in c.lower():
                return c
    return None

def explode_semicol_series(df, col):
    s = (df[col].astype(str)
                  .where(lambda x: x.str.lower().ne('nan'))
                  .str.split(';'))
    s = s.explode().dropna().astype(str).str.strip()
    return s[s != ""]

def top_n_counts(series, n=10):
    vc = series.value_counts().reset_index()
    vc.columns = ["Item", "Count"]
    return vc.head(n)

# ------- carga -------
df = pd.read_csv(INPUT_CSV, low_memory=False)
df.columns = df.columns.str.strip()

# Escoge qué categorías quieres generar (las 4 actuales del lab)
CATEGORIES_TO_BUILD = [
    "LanguageHaveWorkedWith",
    "DatabaseHaveWorkedWith",
    "PlatformHaveWorkedWith",
    "WebframeHaveWorkedWith",   # <- usa tu nombre real
    # si luego haces la pestaña de "Future Trend", añade los *WantToWorkWith* aquí
    # "LanguageWantToWorkWith", "DatabaseWantToWorkWith",
    # "PlatformWantToWorkWith", "WebframeWantToWorkWith"
]

long_rows = []
for canonical in CATEGORIES_TO_BUILD:
    candidates = ALIASES.get(canonical, [canonical])
    real_col = resolve_col(df, candidates)
    if not real_col:
        print(f"⚠️  No encontré columna para {canonical}. Revisa nombres en tu CSV.")
        continue
    ser = explode_semicol_series(df, real_col)
    if ser.empty:
        print(f"⚠️  Columna {real_col} está vacía tras limpiar/separar.")
        continue

    top10 = top_n_counts(ser, n=10)
    # guarda CSV por categoría
    safe_name = canonical.replace("HaveWorkedWith","").replace("WantToWorkWith","").replace("Webframe","WebFrame")
    top10.to_csv(OUT_DIR / f"Top10_{safe_name}.csv", index=False, encoding="utf-8")
    print(f"✅ Top10_{safe_name}.csv (columna real usada: {real_col})")

    # agrega al consolidado largo
    t = top10.copy()
    t.insert(0, "Category", canonical)   # Category | Item | Count
    long_rows.append(t[["Category","Item","Count"]])

# consolidado largo (1 solo CSV para Cognos)
if long_rows:
    long_df = pd.concat(long_rows, ignore_index=True)
    long_df.to_csv(OUT_DIR / "Top10_AllCategories_Long.csv", index=False, encoding="utf-8")
    print("✅ Top10_AllCategories_Long.csv listo.")
else:
    print("⚠️ No se generó consolidado; revisa nombres de columnas.")


✅ Top10_Language.csv (columna real usada: LanguageHaveWorkedWith)
✅ Top10_Database.csv (columna real usada: DatabaseHaveWorkedWith)
✅ Top10_Platform.csv (columna real usada: PlatformHaveWorkedWith)
✅ Top10_WebFrame.csv (columna real usada: WebframeHaveWorkedWith)
✅ Top10_AllCategories_Long.csv listo.


In [3]:
print(list(df.columns))

['ResponseId', 'MainBranch', 'Age', 'Employment', 'RemoteWork', 'Check', 'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline', 'TechDoc', 'YearsCode', 'YearsCodePro', 'DevType', 'OrgSize', 'PurchaseInfluence', 'BuyNewTool', 'BuildvsBuy', 'TechEndorse', 'Country', 'Currency', 'CompTotal', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'LanguageAdmired', 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith', 'DatabaseAdmired', 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'PlatformAdmired', 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith', 'WebframeAdmired', 'EmbeddedHaveWorkedWith', 'EmbeddedWantToWorkWith', 'EmbeddedAdmired', 'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'MiscTechAdmired', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith', 'ToolsTechAdmired', 'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', 'NEWCollabToolsAdmired', 'OpSysPersonal use', 'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWantToWorkWith',